In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('Train_Loan_Home.csv')

In [3]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
columnsX = data.columns[1:-1]
columnsY = data.columns[-1]

In [6]:
columnsY

'Loan_Status'

In [14]:
catCol = ['Gender','Married','Education','Self_Employed','Property_Area','Dependents']
numWithScalingCol = ['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term']
numCol = ['Credit_History']

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn import ensemble

In [16]:
data.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [17]:
numeric_transformer_scaling = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value=99))])
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer_scaling, numWithScalingCol),
    ('num2', numeric_transformer, numCol),
    ('cat', categorical_transformer, catCol)
])

In [18]:
modelPipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                ('classifier', ensemble.RandomForestClassifier())])

In [19]:
modelPipeline.fit(data[columnsX], data[columnsY])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['ApplicantIncome',
                                                   'CoapplicantIncome',
                                                   'LoanAmount',
                                                   'Loan_Amount_Term']),
                                                 ('num2',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=99,
                                                     

In [20]:
scores = modelPipeline.predict_proba(data[columnsX])[:,-1]

In [21]:
scores

array([0.96, 0.3 , 0.93, 0.93, 0.99, 0.94, 0.91, 0.08, 0.91, 0.14, 0.93,
       0.91, 0.91, 0.27, 0.85, 0.97, 0.86, 0.06, 0.22, 0.92, 0.04, 0.94,
       0.12, 0.08, 0.26, 0.92, 1.  , 0.91, 0.17, 0.93, 0.22, 0.33, 0.34,
       0.99, 0.24, 0.96, 0.05, 1.  , 0.84, 0.99, 0.28, 0.93, 0.91, 1.  ,
       0.79, 0.97, 0.9 , 0.95, 0.05, 0.9 , 0.98, 0.98, 0.37, 0.23, 0.07,
       0.99, 0.99, 0.27, 0.94, 0.92, 0.95, 0.92, 0.12, 0.03, 0.02, 0.34,
       0.06, 0.88, 0.82, 0.06, 0.91, 0.91, 0.95, 0.19, 0.96, 0.19, 0.4 ,
       0.2 , 0.08, 0.86, 0.93, 0.96, 0.19, 0.29, 0.85, 0.95, 0.95, 1.  ,
       0.78, 0.98, 1.  , 0.93, 0.95, 0.86, 0.9 , 0.32, 0.98, 0.98, 0.93,
       0.92, 0.88, 0.92, 0.91, 0.97, 0.92, 0.98, 0.97, 0.28, 0.05, 0.91,
       0.99, 0.98, 0.06, 0.93, 0.96, 0.9 , 1.  , 0.98, 0.31, 0.83, 0.82,
       0.97, 0.67, 0.98, 0.88, 0.95, 0.86, 0.99, 0.09, 0.27, 0.71, 0.8 ,
       0.99, 0.96, 0.98, 0.28, 0.21, 0.93, 0.12, 0.28, 0.42, 0.96, 0.9 ,
       1.  , 0.99, 0.97, 0.82, 0.89, 0.23, 1.  , 0.

In [22]:
data2 = pd.read_csv('Test_Loan_Home.csv')

In [23]:
scores = modelPipeline.predict_proba(data2[columnsX])[:,-1]

In [ ]:
scores